In [1]:
include("src\\ImportanceSampling.jl")
using ImportanceSampling
using Distributions

In [2]:
g!s = [((r, x) -> r .= 2*x, [1.0])
    ((r, x) -> begin
        y = first(x)
        r[1] = 5*y.^2
        r[2] = 2*y.^3
        r
    end, [5/3, 1/2])]

f!(r, x) = r .= x.^2 - 3*x

f! (generic function with 1 method)

In [3]:
is = CvImportanceSampler(f!, 1, g!s, Uniform(), w=((_)->1.0), use_q=false)

ImportanceSampling.CvImportanceSampler(Distributions.Uniform{Float64}(a=0.0, b=1.0), ImportanceSampling.MeanVariance{1}(0, [0.0], [0.0]), ImportanceSampling.Diagnostic(0, 0.0, 0.0, 0.0, 0.0), ImportanceSampling.ControlVariate{1,3}(0, [0.0; 0.0; 0.0], [0.0], [0.0, 0.0, 0.0], [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0], [0.0; 0.0; 0.0]), #11, f!, Function[#1, #2], Array{Float64,1}[[1.0], [1.66667, 0.5]])

In [4]:
batchsize = 10000
X = Matrix{Float64}(length(is.q), batchsize)
F = Matrix{Float64}(length(is.μ), batchsize)
G = Matrix{Float64}(size(is.β, 2), batchsize)
W = Vector{Float64}(batchsize)
Q = Vector{Float64}(batchsize)
;

In [5]:
rand!(is.q, X)
ImportanceSampling.calcF!(F, is, X)
ImportanceSampling.calcG!(G, is, X)
ImportanceSampling.calcW!(W, is, X)
ImportanceSampling.calcQ!(Q, is, X)
;

In [6]:
updateβ!(is, X=X, F=F, G=G, W=W, Q=Q)

ImportanceSampling.CvImportanceSampler(Distributions.Uniform{Float64}(a=0.0, b=1.0), ImportanceSampling.MeanVariance{1}(0, [0.0], [0.0]), ImportanceSampling.Diagnostic(0, 0.0, 0.0, 0.0, 0.0), ImportanceSampling.ControlVariate{1,3}(10000, [-1.5; 0.2; -2.31178e-13], [-11627.3], [9970.13, 16639.5, 5000.99], [3371.22 8415.2 3029.17; 8415.2 22408.0 8401.52; 3029.17 8401.52 3239.75], [-3373.78; -8141.2; -2863.46]), #11, f!, Function[#1, #2], Array{Float64,1}[[1.0], [1.66667, 0.5]])

In [13]:
updateμ!!(is, niters=1000, nbatches=1)#, updateβ=true, updateμ=false)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] pdf!(::Array{Float64,1}, ::Distributions.Uniform{Float64}, ::Array{Float64,2}) at .\deprecated.jl:57
 [3] #_update!#41(::Void, ::Void, ::Bool, ::Bool, ::Array{Any,1}, ::Function, ::Val{true}, ::ImportanceSampling.CvImportanceSampler, ::Void, ::Void, ::Void, ::Int64, ::Int64, ::Int64) at C:\Users\Hamza El-Saawy\Documents\school\stats362\project\src\importance_sampler.jl:345
 [4] (::ImportanceSampling.#kw##_update!)(::Array{Any,1}, ::ImportanceSampling.#_update!, ::Val{true}, ::ImportanceSampling.CvImportanceSampler, ::Void, ::Void, ::Void, ::Int64, ::Int64, ::Int64) at .\<missing>:0
 [5] #update!#13(::Void, ::Void, ::Void, ::Int64, ::Int64, ::Int64, ::Array{Any,1}, ::Function, ::ImportanceSampling.CvImportanceSampler) at C:\Users\Hamza El-Saawy\Documents\school\stats362\project\src\importance_sampler.jl:63
 [6] (::ImportanceSampling.#kw##update!)(::Array{Any,1}, ::ImportanceSampling.#update!, ::ImportanceSampling.Cv

LoadError: [91mDimensionMismatch("cannot broadcast array to have fewer dimensions")[39m